# SPRINT 12: ML: MODELOS SUPERVISADOS III. OTROS MODELOS Y REPASO

## K-NEAREST NEIGHBOURS (KNN) - Clasificación y Regresión

KNN es un algoritmo de aprendizaje supervisado basado en la proximidad entre vectores. No requiere entrenamiento tradicional: simplemente almacena los datos y compara distancias al hacer predicciones.

🧠 Fundamento
- Clasificación: Asigna una clase en base a la moda (mayoría) de los vecinos más cercanos.

- Regresión: Predice un valor promedio de los vecinos más cercanos.

🔁 Pasos para aplicar KNN (clasificación o regresión)

In [ ]:
# 1. Cargar y preparar datos

import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Escalar variables (muy importante en KNN)
# KNN es muy sensible a la escala, así que es necesario normalizar 
# o estandarizar.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Importar e instanciar el modelo

from sklearn.neighbors import KNeighborsClassifier  # o KNeighborsRegressor
knn = KNeighborsClassifier(n_neighbors=K)  # elegir K según análisis

# 4. Entrenar el modelo (aunque KNN "entrena" simplemente almacenando dataset)

knn.fit(X_train, y_train)

# 5. Realizar predicciones

y_pred = knn.predict(X_test)

# 6. Evaluar el modelo
# Clasificación: accuracy, matriz de confusión, F1...

from sklearn.metrics import accuracy_score, confusion_matrix

# Regresión: MAE, RMSE, R2

from sklearn.metrics import mean_squared_error

⚙️ Selección del valor óptimo de K
- No hay un método exacto. Lo más habitual es probar varios valores (ej: de 1 a 20) y elegir el que maximice el rendimiento.

- Valores pequeños → muy sensibles al ruido.

- Valores grandes → modelos más estables pero menos precisos en clases minoritarias.

Ejemplo de búsqueda de K óptimo:

In [ ]:
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print(f"K={k} → Accuracy: {knn.score(X_test, y_test)}")

📎 Notas clave
- KNN es lento con muchos datos, porque calcula distancias con todos los puntos.

- Puede usarse tanto para problemas de clasificación como de regresión.

- Requiere preprocesamiento cuidadoso (escalado de datos).

## 📌 Regresión Polinómica (Polynomial Regression)

Cuando la relación entre variables no es lineal, una regresión lineal no es suficiente. La regresión polinómica permite capturar relaciones no lineales mediante la transformación de las variables originales en nuevas variables elevadas a potencias.

🧠 Fundamento
Consiste en ampliar el espacio de variables incluyendo:

Potencias de las features originales: 
𝑥
,
𝑥
2
,
𝑥
3
,
…
x,x 
2
 ,x 
3
 ,…

Combinaciones entre variables (si hay más de una feature)

Aunque el modelo sigue siendo lineal en los coeficientes, el comportamiento se vuelve no lineal respecto a las variables.

🔁 Pasos para aplicar regresión polinómica

In [ ]:
# 1. Preparar los datos
from sklearn.model_selection import train_test_split
X = df[["feature"]].values
y = df["target"].values

# 2. Transformar las variables con PolynomialFeatures

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=d)  # d es el grado del polinomio
X_poly = poly.fit_transform(X)

# 3. Entrenar el modelo de regresión lineal

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_poly, y)

# 4. Predecir nuevos valores
#Recordar: hay que transformar las features antes de predecir

X_test_poly = poly.transform(X_test)
y_pred = model.predict(X_test_poly)

⚠️ Sobreajuste (Overfitting)
Grados altos capturan muy bien el patrón del entrenamiento, pero generalizan mal.

Es recomendable:

- Usar validación cruzada para elegir el grado.

- Visualizar el error en training vs test.

- No pasar de grado 3-4 sin justificación sólida.

## 📌 Support Vector Machine (SVM)

SVM es un modelo supervisado de clasificación (también puede usarse en regresión). Su objetivo es encontrar el hiperplano que mejor separa las clases maximizando el margen entre los puntos de distintas clases.

🧠 Fundamento
- Busca el separador lineal con mayor margen entre las clases.

- Los puntos más cercanos al hiperplano se llaman vectores de soporte, y determinan el modelo.

- Si los datos no son separables linealmente, se recurre a kernels para transformar el espacio.

🔁 Pasos para aplicar un modelo SVM (clasificación)

In [ ]:
# 1. Preparar los datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# 2. Escalado obligatorio
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Instanciar y entrenar el modelo
from sklearn.svm import SVC
model = SVC(kernel='linear', C=1)  # 'linear', 'poly', 'rbf'
model.fit(X_train, y_train)

# 4. Predecir y evaluar
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

🔀 Parámetros importantes
- kernel: tipo de transformación (ver más abajo).

- C: penalización por errores (cuanto mayor, menos margen permite → modelo más estricto).

- gamma: en kernels RBF, controla cuánto influye un solo punto (bajo = amplio, alto = localizado).

🧪 Kernels: separación no lineal

Cuando los datos no son linealmente separables, se usan kernels que transforman el espacio:
| Kernel   | Descripción                              |
| -------- | ---------------------------------------- |
| `linear` | Hiperplano lineal                        |
| `poly`   | Polinómico (grado definido con `degree`) |
| `rbf`    | Radial (transforma a dimensión infinita) |


In [ ]:
SVC(kernel='poly', degree=3)
SVC(kernel='rbf', gamma=0.1)

## 📉 SVM en regresión (SVR)

También puede usarse para regresión, con SVR:

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf')
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

⚠️ Consideraciones clave
- Muy sensible al escalado de los datos. Siempre escalar antes.

- Sensibilidad a outliers: puede alterar el margen y los vectores de soporte.

- No es recomendable para datasets grandes, ya que el entrenamiento puede ser costoso.

- Hiperparámetro C controla el trade-off entre margen amplio y clasificación perfecta.

✅ Guía paso a paso para un modelo supervisado con KNN


In [ ]:
# 1. Carga de datos
df = pd.read_csv("ruta_al_csv", sep=";")
df.head()
df.info()

# 2. Análisis del target
bt.pinta_distribucion_categoricas(df, [target], mostrar_valores=True, relativa=True)

#🔹 Si está desbalanceado, apúntalo. Esto afectará al rendimiento de modelos 
# y puede requerir:
# Usar métricas robustas (recall, balanced accuracy)
# Hacer balanceo (SMOTE, undersampling...) en casos graves

# 3. Limpieza de columnas no útiles (irrelevantes o con muchos nulos)
df.drop("columna_con_nulos", axis=1, inplace=True)

# 4. División Train/Test
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# 5. Análisis de variables numéricas
features_num = ["var1", "var2"]
train_set[features_num].describe()
train_set[features_num].hist()

#🔸 Revisa:
# Rango entre variables → escalado necesario para KNN
# Distribución sesgada → aplica log si hay mucha asimetría

# 6. Análisis de variables categóricas
features_cat = ["col_categorica"]
bt.pinta_distribucion_categoricas(train_set, features_cat, mostrar_valores=True, relativa=True)

#✔️ Si es binaria, puedes codificarla como 0 y 1
#✔️ Si tiene más de dos categorías, necesitarás one-hot encoding (para otros modelos)

#7. Preprocesamiento de features

# Categórica → binaria
train_set["col"] = train_set["col"].apply(lambda x: 1 if x == "positive" else 0)
test_set["col"] = test_set["col"].apply(lambda x: 1 if x == "positive" else 0)

# Numéricas → escalado
scaler = MinMaxScaler()
train_set[features_num] = scaler.fit_transform(train_set[features_num])
test_set[features_num] = scaler.transform(test_set[features_num])

# 8. Separar X e y y entrenar KNN
X_train = train_set[features_cat + features_num]
y_train = train_set[target]
X_test = test_set[features_cat + features_num]
y_test = test_set[target]

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# 9. Evaluación inicial
print(classification_report(y_train, knn.predict(X_train)))
print(classification_report(y_test, knn.predict(X_test)))

# 10. Optimizar K manualmente
metricas = []
for k in range(1, 21):
    model = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="balanced_accuracy").mean()
    metricas.append(score)

best_k = np.argmax(metricas) + 1

#✔️ Usa balanced_accuracy si el dataset está desbalanceado
#✔️ Entrena y evalúa el modelo final con el mejor k

# 11. Búsqueda de hiperparámetros con GridSearch
param_grid = {
    "n_neighbors": range(1, 20),
    "weights": ["uniform", "distance"]
}

grid = GridSearchCV(knn, param_grid, cv=5, scoring="balanced_accuracy")
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_score_)
print(classification_report(y_test, grid.best_estimator_.predict(X_test)))

# ✔️ weights="distance" puede mejorar si hay ruido o vecinos poco representativos




## ⚖️ Equilibrado de Clases (Class Imbalance)
📌 ¿Qué es un dataset desbalanceado?
Un dataset está desbalanceado cuando las clases del target no están representadas de forma proporcional, es decir, una clase aparece con mucha más frecuencia que otra.
Ejemplo típico: 90% clase 'no', 10% clase 'yes'.

❗¿Por qué es un problema?
Los modelos pueden aprender a predecir siempre la clase mayoritaria y tener una alta accuracy pero bajo recall o F1-score para la clase minoritaria.

💡 Consejo inicial:
Antes de aplicar técnicas, entiende bien el objetivo de negocio:

- ¿Interesa maximizar el recall de la clase minoritaria?

- ¿O reducir los falsos positivos?

- ¿Qué cuesta y qué se gana con un TP, FP, FN, TN?

### 🧪 Estrategias para abordar el desequilibrio

✅ 0. No hacer nada (modelo base)
Evaluar sin modificar el dataset:

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

Métricas clave:

- Precisión / recall de la clase minoritaria

- Matriz de confusión: ConfusionMatrixDisplay.from_predictions(...)

🔁 1. Over-sampling (sobremuestreo)

🧬 SMOTE (Synthetic Minority Over-sampling Technique)

Crea nuevas observaciones sintéticas de la clase minoritaria usando KNN.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

Pros: mejora el recall sin perder datos
Contras: riesgo de overfitting si se abusa

📦 Otras alternativas de oversampling:

- RandomOverSampler() (repite observaciones)

- ADASYN() (parecido a SMOTE, pero adaptativo)

🔻 2. Under-sampling (bajomuestreo)
Reduce el número de observaciones de la clase mayoritaria.

In [ ]:
from sklearn.utils import resample

may = X_train[y_train == "no"]
min_ = X_train[y_train == "yes"]

may_down = resample(may, replace=False, n_samples=len(min_), random_state=42)

X_train_bal = pd.concat([may_down, min_])
y_train_bal = pd.concat([y_train.loc[may_down.index], y_train.loc[min_.index]])

Pros: rápido y simple
Contras: pierde datos valiosos → riesgo de subentrenamiento

⚖️ 3. Ajuste de pesos (class_weight)

Aplica un peso mayor a la clase minoritaria en la función de pérdida del modelo. Recomendado si no quieres tocar el dataset.

In [ ]:
model = LogisticRegression(class_weight='balanced')

✔️ Muchos modelos de sklearn lo soportan (DecisionTree, SVM, RandomForest, etc.)

📊 Comparativa de resultados
| Técnica            | Precisión (YES) | Recall (YES) | Comentario                                         |
| ------------------ | --------------- | ------------ | -------------------------------------------------- |
| **Sin equilibrar** | 0.64            | 0.36         | Alta precisión clase NO, muy bajo recall clase YES |
| **SMOTE**          | 0.51            | 0.61         | Mejora de recall, algo de precisión                |
| **Under-sampling** | 0.39            | 0.85         | Mucho recall, baja precisión (dispara a todo)      |
| **class\_weight**  | 0.39            | 0.85         | Similar a under-sampling, pero sin eliminar datos  |

### 🧠 Consejos finales para el equilibrio de clases

⚠️ No hay una solución universal → prueba varias y evalúa

📉 Siempre evalúa con métricas por clase: precisión, recall, F1

📊 Usa validación cruzada para testear robustez

🎯 Ajusta el umbral de decisión (predict_proba) si no quieres reequilibrar el dataset

-------------

# 🧠 Análisis de Errores

Analizar los errores es esencial para mejorar un modelo y adaptarlo a las prioridades reales del negocio. No basta con obtener un buen accuracy global: es necesario entender qué está fallando, cómo y por qué.

## ⚠️ Importancia del Análisis de Errores
- Identifica clases con bajo rendimiento (recall, precisión...).

- Detecta patrones sistemáticos de error.

- Prioriza mejoras según el impacto del fallo (no todas las clases pesan igual).

- Te guía hacia una mejor interpretación del modelo y ajustes más efectivos.

## 🔍 Análisis de errores en Clasificación
✅ Proceso paso a paso



In [ ]:
# 1. Entrena un modelo de clasificación

# 2. Evalúa con métricas por clase

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

# 3. Inspecciona la matriz de confusión normalizada (recall por fila)

from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize='true')

## 🧭 Cómo interpretar la matriz
- Lectura por filas (con normalize="true"):
Cada fila representa una clase real → el valor muestra a qué clase fue clasificada.

- Las diagonales altas indican buen recall.

- Las desviaciones fuera de la diagonal indican confusiones sistemáticas.

💡 Ejemplo de análisis cualitativo

- Si clase 2 se clasifica mal y casi siempre como clase 4 o 5 → hay un sesgo ascendente.

- Si una clase con poco soporte tiene bajo recall, puedes:

    -Reentrenar un modelo específico para esa clase.

    - Balancear las clases (oversampling).

    - Aplicar un modelo de segundo nivel ("modelo cascada").


## 📈 Sugerencias para mejorar tras el análisis
1. Ingeniería de variables
Nuevas variables o transformar las existentes (escalado, codificación ordinal...).

2. Modelos especializados
Para clases con errores sistemáticos o de alto impacto → crear modelos dedicados.

3. Reentrenar con class_weight
Penalizar más los errores en las clases minoritarias.

4. Evaluar impacto real
Discutir con negocio: ¿cuál es el coste real de predecir mal cada clase?

## 📉 Análisis de Errores en Regresión
Cuando trabajamos con modelos de regresión, no podemos usar una matriz de confusión como en clasificación, pero sí podemos analizar los errores individuales (residuos), su distribución y su relación con las predicciones. Esto nos ayuda a identificar sesgos o zonas problemáticas.

⚙️ Preparación del problema de regresión

Se cambia el target a una variable numérica continua. En el ejemplo con el dataset diamonds:

In [ ]:
target_regresion = "price"
features_cat_reg = ["color", "clarity", "cut"]
features_num_reg = [...]  # numéricas sin incluir "price"

Se aplica StandardScaler a las numéricas y luego se invierte el escalado del target para que vuelva a su escala real:

In [ ]:
train_set["price"] = scaler.inverse_transform(train_set[features_num])[..., price_idx]

### 🧪 Métricas de evaluación en regresión
| Métrica                                  | Interpretación                                                        |
| ---------------------------------------- | --------------------------------------------------------------------- |
| `MAE` (Error absoluto medio)             | Cuánto se equivoca el modelo de media (sin penalizar grandes errores) |
| `MAPE` (Error porcentual medio)          | Cuánto se equivoca el modelo en proporción al valor real              |
| `RMSE` (Raíz del error cuadrático medio) | Penaliza más los errores grandes                                      |


### 📊 Visualizaciones esenciales
1. Gráfico de reales vs predichos

Permite detectar si el modelo funciona peor en ciertos rangos del target (por ejemplo, en precios altos):

In [ ]:
plot_predictions_vs_actual(y_real, y_pred)

Idealmente, los puntos deben estar alineados con la línea roja y = x.

2. Distribución de residuos

Ayuda a ver si los errores tienen una distribución simétrica o sesgada:

In [ ]:
residuos = y_test - y_pred
sns.histplot(residuos, kde=True)

Una distribución normal de residuos suele indicar un modelo razonablemente ajustado.

3. Residuos vs Predicciones

Permite identificar zonas del rango de predicción donde el modelo sesga (por ejemplo, subestimando sistemáticamente):

In [ ]:
plt.scatter(y_pred, residuos)

Un buen modelo debería mostrar los residuos dispersos alrededor del 0 sin patrones claros.

### 🧠 ¿Qué hacer con la información del análisis?

En el ejemplo de diamonds, se observa que:

El modelo funciona mejor en precios bajos.

Para precios > 7500, hay mayor error y dispersión.

El modelo parece sesgar más en la "cola larga" de la distribución.

Esto puede indicar que:

El modelo necesita mayor capacidad para manejar precios altos.

Puede ser útil hacer un modelo específico para ese segmento.

Otra opción es binnear el target y convertirlo en clasificación.

### 💡 Sugerencias de mejora

1. Ingeniería de características

Transformar o crear nuevas features que ayuden al modelo.

2. Modelos alternativos

RandomForest, GradientBoosting, XGBoost...

3. Ajuste de hiperparámetros

Profundidad, número de árboles, learning rate...

4. Tratamiento de outliers

Revisar los valores extremos que distorsionan la predicción.

5. Segmentación del modelo

Crear un modelo exclusivo para valores altos o usar ensemble de modelos.